1. PARSE XML FILES
2. TURN INTO TREE
3. IF FAILS, PARSE AS MANY OF THE ENTRIES AS POSSIBLE

In [2]:
import traceback
import xml.etree.ElementTree as ET
from lxml import etree as ET2
from pathlib import Path
import re
import shutil
import os

import git

repo = git.Repo('.', search_parent_directories=True)
os.chdir(repo.working_tree_dir)

srcpaths=(
    "first_run_xmls/logs-xmls",
)

xmls = []
for d in srcpaths:
    l = list(Path(d).glob("*.xml"))
    l = [p for p in l if ".repair." not in p.name]
    xmls += l
print(len(xmls), "XML files found,", len(set([f.name for f in xmls])), "unique. duplicates =", [f for f in xmls if [g.name for g in xmls].count(f.name) > 1])

185 XML files found, 185 unique. duplicates = []


In [6]:
dstdir=Path("postprocessed-xmls-lxml")

In [7]:
if dstdir.exists():
    shutil.rmtree(dstdir)
dstdir.mkdir(exist_ok=True)

In [15]:
import tqdm as tqdm
import copy
from multiprocessing import Pool

def convert_file(fpath, dstdir):
    with open(dstdir/fpath.name, "w") as of:
        print("<repaired-trace>", file=of)
        try:
            it = ET2.iterparse(fpath)
            it = (n for _, n in it if n.tag == "call")
            for n in it:
                n = copy.deepcopy(n)
                n.attrib["xml"] = str(fpath)
                for c in n:
                    if c.tag == "call":
                        n.remove(c)
                print(ET2.tostring(n), file=of)
                del n
            return "success"
        except ET.ParseError as ex:
            return "failure"
        except ET2.XMLSyntaxError as ex:
            return "failure"
        finally:
            print("</repaired-trace>", file=of)

successes_1 = []
failed_1 = []
with Pool(8) as pool:
    with tqdm.tqdm(pool.imap(convert_file, xmls, dstdir=dstdir)) as pbar:
        for fpath in pbar:
            successes_1.append(fpath)
            failed_1.append(fpath)
            pbar.set_postfix({
                "success": len(successes_1),
                "failed": len(failed_1),
            })

print("ROUND 1:", len(failed_1), "files failed parsing")
(dstdir/"1_repair_success_1.txt").write_text("\n".join(map(str, successes_1)))

  0%|          | 0/185 [00:08<?, ?it/s]


KeyboardInterrupt: 